## Imports

In [1]:
import json
import time
import textwrap
import dotenv

## Loading the LLM

In [3]:
dotenv.load_dotenv()
from langchain_openai import ChatOpenAI, AzureChatOpenAI

llm = AzureChatOpenAI(
    max_tokens=16000,
    temperature=0.5,
    deployment_name="gpt-4o-b",
    model_name="gpt-4o",
)
print(f'Using Azure endpoint: {llm.azure_endpoint}')

# Test if LLM responds OK
prompt = "What is your training data cutoff date?"
response = llm(prompt)
print(response.content)

Using Azure endpoint: https://summarilyazureopenai.openai.azure.com


/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_16438/663203810.py:14: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)


My training data goes up until **October 2021**. I don't have knowledge of events or developments that occurred after that date. Let me know how I can assist you!


## Agent #1 : 
**Tools: Calculator and Wikipedia API call**

**Note:** It's important to set temperature to 0.0 because we are going to use LLM as a reasoning engine of our agent. 

In [11]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities import PythonREPL

In [14]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

**Explanation:** 

- `llm-math` is a chain itself which uses llm in conjunction with a calculator to do the math operation. 
- `wikipedia` tool is an API that connects to the Wikipedia allowing you to run search queries and get results.

In [18]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)

In [16]:
agent("what is the result of 5 + 6")

/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_16438/1104946195.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agents("what is the result of 5 + 6")




> Entering new AgentExecutor chain...
Thought: This is a simple arithmetic problem that can be solved directly.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The result of 5 + 6 is 11."
}
```

> Finished chain.


{'input': 'what is the result of 5 + 6',
 'output': 'The result of 5 + 6 is 11.'}

In [17]:
agent("what is 25% of 300")



> Entering new AgentExecutor chain...
Thought: This is a straightforward percentage calculation. I will compute 25% of 300 using the calculator tool.
Action:
```
{
  "action": "Calculator",
  "action_input": {"tool_input": "25/100 * 300"}
}
```
Observation: Answer: 75.0
Thought:I know what to respond. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "25% of 300 is 75."
}
```

> Finished chain.


{'input': 'what is 25% of 300', 'output': '25% of 300 is 75.'}

In [19]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should look up Tom M. Mitchell and his authored works to confirm the book he wrote. 
Action:
```
{
  "action": "wikipedia",
  "action_input": {
    "query": "Tom M. Mitchell"
  }
}
```

/Users/mehdiazad/Documents/Projects/mini_projects/miniproj/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/mehdiazad/Documents/Projects/mini_projects/miniproj/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Oren Etzioni
Summary: Oren Etzioni (born 1964)  is Professor Emeritus of computer science, and founding CEO of the Allen 

### Agent #2: 
**Tools: Python REPL**

In [20]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [21]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [22]:
agent.run(f"sort these customers by \
           first name and last name and print the result: {customer_list} ")

/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_16438/4142660271.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run(f"sort these customers by \




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I need to sort the given list of customers by their first names and last names. To do this, I will use Python's `sorted` function with a custom key that sorts by both the first name and last name. I will then print the sorted result.

Action: Python_REPL
Action Input:
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[0], x[1]))
print(sorted_customers)
```
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought:I now know the final answer.

Final Answer: The customers sorted by first name and last name are: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']].

> Finished chain.


"The customers sorted by first name and last name are: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]."

### Agent #3: 
**Tools: User defined function** 

In [23]:
from langchain.agents import tool
from datetime import date

In [24]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [27]:
agent = initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)

In [28]:
agent("what is the date today?")



> Entering new AgentExecutor chain...
Thought: I need to determine today's date using the appropriate tool.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-01-22
Thought:I now know the final answer.  
Final Answer: Today's date is January 22, 2025.

> Finished chain.


{'input': 'what is the date today?',
 'output': "Today's date is January 22, 2025."}